In [1]:
import numpy as np
import pandas as pd
import matplotlib as mlt
import matplotlib.pyplot as plt
import seaborn as sns
import re
from urllib.parse import urljoin

from matplotlib import font_manager, rc   
font_name = font_manager.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

pd.options.display.float_format = "{:,.2f}".format

# 1. 공시지가 데이터셋 만들기

In [2]:
income3 = pd.read_csv("data/income3.csv",encoding="euc-kr") #정부공공데이터:encoding="euc-kr"
income3.head()

C:\Users\LG\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (3,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SIDO_NM,SIGUNGU_NM,BJDONG_NM,LAND_CD,JIGA(가격),SGG_CD,BJDONG_CD,LAND_GBN,LAND_GBN_NM,BOBN,BUBN,YEAR,BASE_MON
0,서울특별시,종로구,청운동,1111010100100010000,3765000,11110,10100,1,일반,1,0,2018,2018-01-01
1,서울특별시,종로구,청운동,1111010100100010001,1171000,11110,10100,1,일반,1,1,2018,2018-01-01
2,서울특별시,종로구,청운동,1111010100100010002,3056000,11110,10100,1,일반,1,2,2018,2018-01-01
3,서울특별시,종로구,청운동,1111010100100010003,3978000,11110,10100,1,일반,1,3,2018,2018-01-01
4,서울특별시,종로구,청운동,1111010100100010004,1425000,11110,10100,1,일반,1,4,2018,2018-01-01


In [3]:
# 컬럼이름 변경
income3.rename(columns={
    income3.columns[1]:"구", income3.columns[2]:"동",  income3.columns[4]:"공시지가",
}, inplace=True)

income3.head()

,SIDO_NM,구,동,LAND_CD,공시지가,SGG_CD,BJDONG_CD,LAND_GBN,LAND_GBN_NM,BOBN,BUBN,YEAR,BASE_MON
0,서울특별시,종로구,청운동,1111010100100010000,3765000,11110,10100,1,일반,1,0,2018,2018-01-01
1,서울특별시,종로구,청운동,1111010100100010001,1171000,11110,10100,1,일반,1,1,2018,2018-01-01
2,서울특별시,종로구,청운동,1111010100100010002,3056000,11110,10100,1,일반,1,2,2018,2018-01-01
3,서울특별시,종로구,청운동,1111010100100010003,3978000,11110,10100,1,일반,1,3,2018,2018-01-01
4,서울특별시,종로구,청운동,1111010100100010004,1425000,11110,10100,1,일반,1,4,2018,2018-01-01


In [4]:
# 같은 동끼리 묶기
# 하나라도 묶으면 데이터 프레임
income3_1 = income3.groupby([income3.구,income3.동]).sum()[["공시지가"]]
income3_1.rename(columns={"공시지가":"공시지가합계"}, inplace = True)
income3_1.head()
income3_1.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 467 entries, (강남구, 개포동) to (중랑구, 중화동)
Data columns (total 1 columns):
공시지가합계    467 non-null int64
dtypes: int64(1)
memory usage: 9.0+ KB


In [5]:
income3_2 = income3.groupby([income3.구,income3.동]).mean()[["공시지가"]]
income3_2.rename(columns={"공시지가":"공시지가평균"}, inplace = True)
income3_2.head()

공시지가평균
구   동               
강남구 개포동 3,798,735.08
    논현동 7,357,435.83
    대치동 8,708,890.54
    도곡동 6,423,487.04
    삼성동 9,496,628.22

In [212]:
income3_2.reset_index()

,구,동,공시지가평균
0,강남구,개포동,"3,798,735.08"
1,강남구,논현동,"7,357,435.83"
2,강남구,대치동,"8,708,890.54"
3,강남구,도곡동,"6,423,487.04"
4,강남구,삼성동,"9,496,628.22"
5,강남구,세곡동,"1,349,543.54"
6,강남구,수서동,"1,862,148.29"
7,강남구,신사동,"10,931,272.60"
8,강남구,압구정동,"9,481,798.86"
9,강남구,역삼동,"9,574,575.46"


# 공시지가 평균, 공시지가 합계의 데이터셋 만들기

In [217]:
mi = pd.merge(income3_1,income3_2, on=["구","동"], how='outer') #full outer: 양쪽에 있는것을 다가져옴
print(mi.head())
print(mi.info())
# 동이 467개

              공시지가합계       공시지가평균
구   동                            
강남구 개포동   9040989500 3,798,735.08
    논현동  40988275000 7,357,435.83
    대치동  27328498500 8,708,890.54
    도곡동  10804305200 6,423,487.04
    삼성동  28299952100 9,496,628.22
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 467 entries, (강남구, 개포동) to (중랑구, 중화동)
Data columns (total 2 columns):
공시지가합계    467 non-null int64
공시지가평균    467 non-null float64
dtypes: float64(1), int64(1)
memory usage: 12.6+ KB
None


In [219]:
mi = mi.reset_index()
mi.head()

,index,구,동,공시지가합계,공시지가평균
0,0,강남구,개포동,9040989500,"3,798,735.08"
1,1,강남구,논현동,40988275000,"7,357,435.83"
2,2,강남구,대치동,27328498500,"8,708,890.54"
3,3,강남구,도곡동,10804305200,"6,423,487.04"
4,4,강남구,삼성동,28299952100,"9,496,628.22"


In [267]:
road = pd.DataFrame(mi, columns=["동"])
# print(road.head())
b = []

for i in range(0, 467):
    df2 = re.split(r"\d", str(road.iloc[i]))
    df2 = list(map(lambda x: x.strip(), df2))
    df2 = list(filter(lambda x: x != '', df2))
    df3 = df2[0].replace("동"," ").replace('Name:',"").strip() 
    
    if len(df3) == 3:
        if df3[-1] != '동':
            df3 = df3 + '동'
    if len(df3) == 2:
        if df3[-1] != '동':
            df3 = df3 + '동'
    if len(df3) == 1:
        df3 = df3 + '동'
    b.append(df3)
    
b

['개포동',
 '논현동',
 '대치동',
 '도곡동',
 '삼성동',
 '세곡동',
 '수서동',
 '신사동',
 '압구정동',
 '역삼동',
 '율현동',
 '일원동',
 '자곡동',
 '청담동',
 '강일동',
 '고덕동',
 '길동',
 '둔촌동',
 '명일동',
 '상일동',
 '성내동',
 '암사동',
 '천호동',
 '미아동',
 '번동',
 '수유동',
 '우이동',
 '가양동',
 '개화동',
 '공항동',
 '과해동',
 '내발산동',
 '등촌동',
 '마곡동',
 '방화동',
 '염창동',
 '오곡동',
 '오쇠동',
 '외발산동',
 '화곡동',
 '남현동',
 '봉천동',
 '신림동',
 '광장동',
 '구의동',
 '군자동',
 '능동',
 '자양동',
 '중곡동',
 '화양동',
 '가리봉동',
 '개봉동',
 '고척동',
 '구로동',
 '궁동',
 '신도림동',
 '오류동',
 '온수동',
 '천왕동',
 '항동',
 '가산동',
 '독산동',
 '시흥동',
 '공릉동',
 '상계동',
 '월계동',
 '중계동',
 '하계동',
 '도봉동',
 '방학동',
 '쌍문동',
 '창동',
 '답십리동',
 '신설동',
 '용두동',
 '이문동',
 '장안동',
 '전농동',
 '제기동',
 '청량리동',
 '회기동',
 '휘경동',
 '노량진동',
 '대방동',
 '작동',
 '본동',
 '사당동',
 '상도동',
 '상도동',
 '신대방동',
 '흑석동',
 '공덕동',
 '구수동',
 '노고산동',
 '당인동',
 '대흥동',
 '도화동',
 '교동',
 '마포동',
 '망원동',
 '상수동',
 '상암동',
 '서교동',
 '성산동',
 '신공덕동',
 '신수동',
 '신정동',
 '아현동',
 '연남동',
 '염리동',
 '용강동',
 '중동',
 '창전동',
 '토정동',
 '하중동',
 '합정동',
 '현석동',
 '남가좌동',
 '냉천동',
 '대신동',
 '대현동',
 '미근동',
 '봉원동',
 '북가좌동',
 '북아현

In [268]:
mi['new동'] = b
mi

,index,구,동,공시지가합계,공시지가평균,new동
0,0,강남구,개포동,9040989500,"3,798,735.08",개포동
1,1,강남구,논현동,40988275000,"7,357,435.83",논현동
2,2,강남구,대치동,27328498500,"8,708,890.54",대치동
3,3,강남구,도곡동,10804305200,"6,423,487.04",도곡동
4,4,강남구,삼성동,28299952100,"9,496,628.22",삼성동
5,5,강남구,세곡동,2005421700,"1,349,543.54",세곡동
6,6,강남구,수서동,1469235000,"1,862,148.29",수서동
7,7,강남구,신사동,32159804000,"10,931,272.60",신사동
8,8,강남구,압구정동,1668796600,"9,481,798.86",압구정동
9,9,강남구,역삼동,58749595000,"9,574,575.46",역삼동


In [272]:
mi['new동'] = b
mi = mi.fillna(0)
mi_1 = mi.groupby(['구','new동']).sum()
mi_1

index       공시지가합계        공시지가평균
구   new동                                  
강남구 개포동       0   9040989500  3,798,735.08
    논현동       1  40988275000  7,357,435.83
    대치동       2  27328498500  8,708,890.54
    도곡동       3  10804305200  6,423,487.04
    삼성동       4  28299952100  9,496,628.22
    세곡동       5   2005421700  1,349,543.54
    수서동       6   1469235000  1,862,148.29
    신사동       7  32159804000 10,931,272.60
    압구정동      8   1668796600  9,481,798.86
    역삼동       9  58749595000  9,574,575.46
    율현동      10   1493977000  1,510,593.53
    일원동      11   5389585600  3,727,237.62
    자곡동      12   1851960400  1,449,108.29
    청담동      13  23170785800 10,202,899.96
강동구 강일동      14   1030033200    645,384.21
    고덕동      15   5141908800  1,822,725.56
    길동       16   9432060700  3,729,561.37
    둔촌동      17   5443783400  2,360,703.99
    명일동      18   4846024300  3,975,409.60
    상일동      19   2556474000  2,292,801.79
    성내동      20  20180103000  3,853,370.82
    암사동      21  11876674000  2,588,074.53
    천호동      22  33300046500  3,836,852.92
강북구 미아동      23  40654991800  2,319,034.38
    번동       24   8824583900  1,916,721.09
    수유동      25  30212616520  1,895,753.06
    우이동      26   3437995700  1,266,296.76
강서구 가양동      27   2976784400  2,813,595.84
    개화동      28   1585836100    367,091.69
    공항동      29   7703521800  2,246,579.70
...         ...          ...           ...
중구  신당동     418  32640026500  3,301,641.36
    쌍림동     419   2440832000  4,921,032.26
    예관동     420   2681981000  8,433,902.52
    예장동     421   1188231200  2,517,438.98
    오장동     422   3131072000  6,249,644.71
    을지로동   2982  35948185000 89,787,943.46
    의주로동    861    775530000 10,443,105.59
    인현동     865   9517585000 16,234,397.22
    입정동     434   4035663000 10,089,157.50
    장교동     435    447170000 14,905,666.67
    장충동     873   6182740000  6,946,002.98
    저동      877   4258463000 22,068,646.70
    정동      440   1162493000  5,900,979.70
    주교동     441   6571139000  9,523,389.86
    주자동     442    216425000  3,435,317.46
    중림동     443   3727422500  4,150,804.57
    초동      444   2717982000 10,179,707.87
    충무로동   2235  19733777000 90,508,535.66
    충정로동    450    392674000  7,551,423.08
    태평로동    903   5068007000 34,284,342.77
    필동     1362   6212866400 15,597,150.42
    황학동     456   8753834000  4,663,736.81
    회현동    1374  11216150000 18,712,089.05
    흥인동     460   2117619000  6,495,763.80
중랑구 망우동     461  13310512600  2,136,861.87
    면목동     462  36797724500  2,333,104.52
    묵동      463   9598138300  2,395,940.66
    상봉동     464  11828205300  2,742,454.28
    신내동     465   3560435200  1,342,041.16
    중화동     466  12514668700  2,296,269.49

[361 rows x 3 columns]

In [273]:
mi_1 = mi_1.reset_index()
mi_1.head()

,구,new동,index,공시지가합계,공시지가평균
0,강남구,개포동,0,9040989500,"3,798,735.08"
1,강남구,논현동,1,40988275000,"7,357,435.83"
2,강남구,대치동,2,27328498500,"8,708,890.54"
3,강남구,도곡동,3,10804305200,"6,423,487.04"
4,강남구,삼성동,4,28299952100,"9,496,628.22"


In [ ]:
del mi_1["index"]

In [297]:
mi_1.head()
mi_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361 entries, 0 to 360
Data columns (total 4 columns):
구         361 non-null object
new동      361 non-null object
공시지가합계    361 non-null int64
공시지가평균    361 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 11.4+ KB


# 2. total 데이터셋 

,Unnamed: 0,구,동,소인(외국인),소인(외국인 비율),소인(한국인),소인(한국인 비율),소인수,소인비율,노인수,...,노인(한국인),노인(한국인 비율),노인(외국인),노인(외국인 비율),총 수급자 가구,총 수급자 인원,순수익,재산세,재산세.1,구key
0,0,종로구,사직동,30,11.03,1444,15.13,1474,15.01,"1,301.00",...,"1,298.00",13.60,3.00,1.10,99,121,nan,NaN,NaN,0
1,1,종로구,삼청동,57,24.05,356,12.55,413,13.44,466.00,...,460.00,16.22,6.00,2.53,34,48,"1,506.94","2,255","2,289",0
2,2,종로구,부암동,17,4.38,1534,15.11,1551,14.72,"1,265.00",...,"1,261.00",12.42,4.00,1.03,104,141,"4,312.28","1,592","1,615",0
3,3,종로구,평창동,9,5.84,3139,16.56,3148,16.48,"2,269.00",...,"2,264.00",11.95,5.00,3.25,94,124,"7,798.93","4,576","4,644",0
4,4,종로구,무악동,8,17.78,1693,20.61,1701,20.59,919.00,...,919.00,11.19,0.00,0.00,101,137,"1,744.48","1,194","1,212",0
5,5,종로구,교남동,18,12.41,1807,16.89,1825,16.83,"1,029.00",...,"1,028.00",9.61,1.00,0.69,131,169,"2,242.80","1,791","1,817",0
6,6,종로구,가회동,12,10.26,601,13.57,613,13.48,645.00,...,644.00,14.54,1.00,0.85,45,60,"54,282.86","1,791","1,817",0
7,7,종로구,종로1.2.3.4가동,183,17.56,488,6.58,671,7.93,"1,287.00",...,"1,273.00",17.16,14.00,1.34,627,640,nan,NaN,NaN,0
8,8,종로구,종로5·6가동,49,13.17,442,8.46,491,8.77,780.00,...,779.00,14.91,1.00,0.27,172,216,"16,678.10","4,311","4,375",0
9,9,종로구,이화동,74,13.75,831,10.28,905,10.50,948.00,...,948.00,11.73,0.00,0.00,150,190,"6,705.69","1,791","1,817",0


In [227]:
import re #정규표현식을 쓰려면 re모듈설치

In [257]:
road = pd.DataFrame(total , columns=["동"])
# print(road.head())
a = []

for i in range(0, 424):
    df2 = re.split(r"\d", str(road.iloc[i]))
    df2 = list(map(lambda x: x.strip(), df2))
    df2 = list(filter(lambda x: x != '', df2))
    df3 = df2[0].replace("동"," ").replace('Name:',"").strip() 
    
    if len(df3) == 3:
        if df3[-1] != '동':
            df3 = df3 + '동'
    if len(df3) == 2:
        if df3[-1] != '동':
            df3 = df3 + '동'
    if len(df3) == 1:
        df3 = df3 + '동'
    a.append(df3)
    
a

['사직동',
 '삼청동',
 '부암동',
 '평창동',
 '무악동',
 '교남동',
 '가회동',
 '종로동',
 '종로동',
 '이화동',
 '창신동',
 '창신동',
 '창신동',
 '숭인동',
 '숭인동',
 '청운효자',
 '혜화동',
 '소공동',
 '회현동',
 '명동',
 '필동',
 '장충동',
 '광희동',
 '을지로동',
 '신당동',
 '황학동',
 '중림동',
 '신당동',
 '다산동',
 '약수동',
 '청구동',
 '화동',
 '후암동',
 '용산동',
 '남영동',
 '원효로동',
 '효창동',
 '용문동',
 '이촌동',
 '이촌동',
 '이태원동',
 '이태원동',
 '서빙고동',
 '보광동',
 '청파동',
 '원효로동',
 '한강로동',
 '한남동',
 '왕십리동',
 '마장동',
 '사근동',
 '행당동',
 '행당동',
 '응봉동',
 '금호동',
 '금호동',
 '성수동',
 '성수동',
 '성수동',
 '성수동',
 '송정동',
 '용답동',
 '왕십리도선',
 '금호동',
 '옥수동',
 '화양동',
 '군자동',
 '중곡동',
 '중곡동',
 '중곡동',
 '중곡동',
 '능동',
 '구의동',
 '구의동',
 '구의동',
 '광장동',
 '자양동',
 '자양동',
 '자양동',
 '자양동',
 '회기동',
 '휘경동',
 '휘경동',
 '청량리동',
 '용신동',
 '제기동',
 '전농동',
 '전농동',
 '답십리동',
 '답십리동',
 '장안동',
 '장안동',
 '이문동',
 '이문동',
 '면목동',
 '면목동',
 '면목동',
 '면목동',
 '상봉동',
 '상봉동',
 '중화동',
 '중화동',
 '묵동',
 '묵동',
 '망우동',
 '신내동',
 '신내동',
 '면목본동',
 '면목동',
 '망우본동',
 '돈암동',
 '돈암동',
 '안암동',
 '보문동',
 '정릉동',
 '정릉동',
 '정릉동',
 '정릉동',
 '길음동',
 '길음동',
 '월곡동',
 '월곡동',
 '장위동',
 '장위동'

In [259]:
total['new동'] = a
total=total.fillna(0)
total_1 = total.groupby(['구','new동']).sum()

In [260]:
total_1 = total_1.reset_index()
total_1.head()

,구,new동,Unnamed: 0,소인(외국인),소인(외국인 비율),소인(한국인),소인(한국인 비율),소인수,소인비율,노인수,노인비율,노인(한국인),노인(한국인 비율),노인(외국인),노인(외국인 비율),순수익,구key
0,강남구,개포동,1115,22,23.67,6865,50.26,6887,50.17,"3,325.00",26.83,"3,321.00",26.92,4.00,8.65,"13,618.49",66
1,강남구,논현동,717,56,11.87,4645,20.58,4701,20.45,"3,711.00",16.05,"3,706.00",16.34,5.00,1.00,"89,175.80",44
2,강남구,대치동,1102,21,14.87,20294,69.88,20315,69.55,"5,128.00",17.41,"5,121.00",17.51,7.00,6.03,"180,670.60",66
3,강남구,도곡동,733,20,13.62,12242,41.99,12262,41.85,"4,348.00",15.04,"4,338.00",15.08,10.00,7.45,"168,667.09",44
4,강남구,삼성동,721,71,22.38,8630,37.19,8701,37.04,"3,370.00",15.29,"3,362.00",15.50,8.00,2.53,"193,143.15",44


In [261]:
del total_1["구key"]
del total_1["Unnamed: 0"]
total_1.head()

,구,new동,소인(외국인),소인(외국인 비율),소인(한국인),소인(한국인 비율),소인수,소인비율,노인수,노인비율,노인(한국인),노인(한국인 비율),노인(외국인),노인(외국인 비율),순수익
0,강남구,개포동,22,23.67,6865,50.26,6887,50.17,"3,325.00",26.83,"3,321.00",26.92,4.00,8.65,"13,618.49"
1,강남구,논현동,56,11.87,4645,20.58,4701,20.45,"3,711.00",16.05,"3,706.00",16.34,5.00,1.00,"89,175.80"
2,강남구,대치동,21,14.87,20294,69.88,20315,69.55,"5,128.00",17.41,"5,121.00",17.51,7.00,6.03,"180,670.60"
3,강남구,도곡동,20,13.62,12242,41.99,12262,41.85,"4,348.00",15.04,"4,338.00",15.08,10.00,7.45,"168,667.09"
4,강남구,삼성동,71,22.38,8630,37.19,8701,37.04,"3,370.00",15.29,"3,362.00",15.50,8.00,2.53,"193,143.15"


In [320]:
total_1.to_csv('final/total.csv', encoding="euc-kr")

# 3. acc데이터셋 

In [286]:
acc = pd.read_csv("traffic_acc.csv",encoding="euc-kr") #정부공공데이터:encoding="euc-kr"
acc.head()

,Unnamed: 0,구,동,보행사고발생건수,노인사고발생건수,어린이사고발생건수
0,0,강남구,개포동,3.00,nan,nan
1,1,강남구,논현동,5.00,nan,nan
2,2,강남구,대치동,3.00,nan,nan
3,3,강남구,도곡동,10.00,3.00,3.00
4,4,강남구,삼성동,4.00,nan,nan


In [280]:
road = pd.DataFrame(acc , columns=["동"])
# print(road.head())
c = []

for i in range(0, 193):
    df2 = re.split(r"\d", str(road.iloc[i]))
    df2 = list(map(lambda x: x.strip(), df2))
    df2 = list(filter(lambda x: x != '', df2))
    df3 = df2[0].replace("동"," ").replace('Name:',"").strip() 
    
    if len(df3) == 3:
        if df3[-1] != '동':
            df3 = df3 + '동'
    if len(df3) == 2:
        if df3[-1] != '동':
            df3 = df3 + '동'
    if len(df3) == 1:
        df3 = df3 + '동'
    c.append(df3)
    
c

['개포동',
 '논현동',
 '대치동',
 '도곡동',
 '삼성동',
 '수서동',
 '역삼동',
 '길동',
 '명일동',
 '성내동',
 '암사동',
 '천호동',
 '미아동',
 '번동',
 '수유동',
 '등촌동',
 '방화동',
 '염창동',
 '화곡동',
 '남현동',
 '봉천동',
 '신림동',
 '구의동',
 '자양동',
 '중곡동',
 '가리봉동',
 '개봉동',
 '고척동',
 '구로동',
 '궁동',
 '신도림동',
 '오류동',
 '가산동',
 '독산동',
 '시흥동',
 '공릉동',
 '상계동',
 '월계동',
 '중계동',
 '하계동',
 '도봉동',
 '방학동',
 '쌍문동',
 '창동',
 '답십리동',
 '신설동',
 '용두동',
 '이문동',
 '장안동',
 '전농동',
 '제기동',
 '청량리동',
 '휘경동',
 '노량진동',
 '대방동',
 '사당동',
 '상도동',
 '상도동',
 '흑석동',
 '구수동',
 '노고산동',
 '도화동',
 '교동',
 '망원동',
 '서교동',
 '아현동',
 '연남동',
 '염리동',
 '남가좌동',
 '북가좌동',
 '북아현동',
 '연희동',
 '영천동',
 '현저동',
 '홍은동',
 '홍제동',
 '방배동',
 '서초동',
 '잠원동',
 '금호동',
 '금호동',
 '도선동',
 '마장동',
 '상왕십리',
 '성수동',
 '성수동',
 '옥수동',
 '용답동',
 '응봉동',
 '행당동',
 '홍익동',
 '길음동',
 '돈암동',
 '선동',
 '선동',
 '선동',
 '소문동',
 '소문동',
 '소문동',
 '삼선동',
 '상월곡동',
 '석관동',
 '안암동',
 '장위동',
 '정릉동',
 '종암동',
 '하월곡동',
 '가락동',
 '마천동',
 '석촌동',
 '오금동',
 '잠실동',
 '풍납동',
 '목동',
 '신월동',
 '신정동',
 '당산동',
 '당산동',
 '당산동',
 '대림동',
 '도림동',
 '문래동',
 '신길동',
 '양평동',
 '영등포

In [282]:
acc['new동'] = c
acc=acc.fillna(0)
acc_1 = acc.groupby(['구','new동']).sum()
acc_1

Unnamed: 0  보행사고발생건수  노인사고발생건수  어린이사고발생건수
구   new동                                           
강남구 개포동            0      3.00      0.00       0.00
    논현동            1      5.00      0.00       0.00
    대치동            2      3.00      0.00       0.00
    도곡동            3     10.00      3.00       3.00
    삼성동            4      4.00      0.00       0.00
    수서동            5      7.00      0.00       0.00
    역삼동            6      4.00      0.00       3.00
강동구 고덕동          173      0.00      3.00       0.00
    길동             7     14.00      4.00       0.00
    명일동            8      7.00     10.00       0.00
    성내동            9     22.00      3.00       0.00
    암사동           10      3.00      5.00       0.00
    천호동           11     12.00      4.00       0.00
강북구 미아동           12     69.00     14.00       4.00
    번동            13     18.00      0.00       0.00
    수유동           14     53.00     17.00       0.00
강서구 공항동          174      0.00      3.00       0.00
    등촌동           15      9.00      0.00       0.00
    방화동           16      9.00      4.00       0.00
    염창동           17      7.00      4.00       0.00
    화곡동           18     39.00      7.00       0.00
관악구 남현동           19      7.00      0.00       0.00
    봉천동           20     91.00      8.00       0.00
    신림동           21     53.00     21.00       0.00
광진구 구의동           22      9.00      3.00       0.00
    군자동          175      0.00      4.00       0.00
    자양동           23      3.00      0.00       0.00
    중곡동           24     20.00      5.00       0.00
구로구 가리봉동          25      9.00      3.00       0.00
    개봉동           26     21.00      9.00       0.00
...              ...       ...       ...        ...
은평구 응암동          143     24.00      9.00       0.00
    증산동          144     10.00      0.00       0.00
종로구 견지동          186      0.00      3.00       0.00
    경운동          145      3.00      0.00       0.00
    관수동          146     10.00      0.00       0.00
    관훈동          147      3.00      0.00       0.00
    낙원동          148     12.00      0.00       0.00
    내자동          187      0.00      3.00       0.00
    묘동           149      4.00      0.00       0.00
    세종로동         150      3.00      0.00       0.00
    숭인동          151     21.00      4.00       0.00
    종로동          836     18.00     12.00       0.00
    창신동          155     12.00      4.00       0.00
    효제동          156      4.00      0.00       0.00
중구  남대문로         157      9.00      4.00       0.00
    남창동          158     14.00      0.00       0.00
    봉래동          159     10.00      4.00       0.00
    북창동          190      0.00      3.00       0.00
    산림동          160      4.00      0.00       0.00
    서소문동         161      2.00      0.00       0.00
    신당동          162     32.00      5.00       0.00
    을지로동         327     11.00      0.00       0.00
    의주로동         165      3.00      0.00       0.00
    충무로동         166      4.00      0.00       0.00
중랑구 망우동          167     26.00      8.00       0.00
    면목동          168     10.00     13.00       0.00
    묵동           169      5.00      4.00       0.00
    상봉동          170     15.00      0.00       0.00
    신내동          171      3.00      0.00       0.00
    중화동          172     11.00      3.00       0.00

[174 rows x 4 columns]

In [284]:
acc_1 = acc_1.reset_index()
acc_1.head()

,구,new동,Unnamed: 0,보행사고발생건수,노인사고발생건수,어린이사고발생건수
0,강남구,개포동,0,3.00,0.00,0.00
1,강남구,논현동,1,5.00,0.00,0.00
2,강남구,대치동,2,3.00,0.00,0.00
3,강남구,도곡동,3,10.00,3.00,3.00
4,강남구,삼성동,4,4.00,0.00,0.00


In [285]:
del acc_1["Unnamed: 0"]
acc_1.head()

,구,new동,보행사고발생건수,노인사고발생건수,어린이사고발생건수
0,강남구,개포동,3.00,0.00,0.00
1,강남구,논현동,5.00,0.00,0.00
2,강남구,대치동,3.00,0.00,0.00
3,강남구,도곡동,10.00,3.00,3.00
4,강남구,삼성동,4.00,0.00,0.00


# 4. 공시지가  데이터셋과 total 합치기

In [304]:
Total1= pd.merge(mi_1,total_1, on=["구","new동"], how='outer') #full outer: 양쪽에 있는것을 다가져옴
Total1.head()
Total1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 412 entries, 0 to 411
Data columns (total 18 columns):
구             412 non-null object
new동          412 non-null object
공시지가합계        361 non-null float64
공시지가평균        361 non-null float64
index         244 non-null float64
소인(외국인)       244 non-null float64
소인(외국인 비율)    244 non-null float64
소인(한국인)       244 non-null float64
소인(한국인 비율)    244 non-null float64
소인수           244 non-null float64
소인비율          244 non-null float64
노인수           244 non-null float64
노인비율          244 non-null float64
노인(한국인)       244 non-null float64
노인(한국인 비율)    244 non-null float64
노인(외국인)       244 non-null float64
노인(외국인 비율)    244 non-null float64
순수익           244 non-null float64
dtypes: float64(16), object(2)
memory usage: 61.2+ KB


# 5. 위에 데이터와 사고 데이터셋 합치기

In [321]:
final= pd.merge(Total1,acc_1, on=["구","new동"], how='outer') #full outer: 양쪽에 있는것을 다가져옴
final.head()
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 412 entries, 0 to 411
Data columns (total 21 columns):
구             412 non-null object
new동          412 non-null object
공시지가합계        361 non-null float64
공시지가평균        361 non-null float64
index         244 non-null float64
소인(외국인)       244 non-null float64
소인(외국인 비율)    244 non-null float64
소인(한국인)       244 non-null float64
소인(한국인 비율)    244 non-null float64
소인수           244 non-null float64
소인비율          244 non-null float64
노인수           244 non-null float64
노인비율          244 non-null float64
노인(한국인)       244 non-null float64
노인(한국인 비율)    244 non-null float64
노인(외국인)       244 non-null float64
노인(외국인 비율)    244 non-null float64
순수익           244 non-null float64
보행사고발생건수      174 non-null float64
노인사고발생건수      174 non-null float64
어린이사고발생건수     174 non-null float64
dtypes: float64(19), object(2)
memory usage: 70.8+ KB


In [322]:
del final["index"]

In [323]:
# null값은 무엇인가..??
final_na_sum = final.isnull().sum()
print(final_na_sum)

구               0
new동            0
공시지가합계         51
공시지가평균         51
소인(외국인)       168
소인(외국인 비율)    168
소인(한국인)       168
소인(한국인 비율)    168
소인수           168
소인비율          168
노인수           168
노인비율          168
노인(한국인)       168
노인(한국인 비율)    168
노인(외국인)       168
노인(외국인 비율)    168
순수익           168
보행사고발생건수      238
노인사고발생건수      238
어린이사고발생건수     238
dtype: int64


In [324]:
final.to_csv('final/final.csv', encoding="euc-kr")

In [325]:
final_in= pd.merge(Total1,acc_1, on=["구","new동"]) #full outer: 양쪽에 있는것을 다가져옴
final_in.head()
final_in.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 0 to 173
Data columns (total 21 columns):
구             174 non-null object
new동          174 non-null object
공시지가합계        174 non-null float64
공시지가평균        174 non-null float64
index         138 non-null float64
소인(외국인)       138 non-null float64
소인(외국인 비율)    138 non-null float64
소인(한국인)       138 non-null float64
소인(한국인 비율)    138 non-null float64
소인수           138 non-null float64
소인비율          138 non-null float64
노인수           138 non-null float64
노인비율          138 non-null float64
노인(한국인)       138 non-null float64
노인(한국인 비율)    138 non-null float64
노인(외국인)       138 non-null float64
노인(외국인 비율)    138 non-null float64
순수익           138 non-null float64
보행사고발생건수      174 non-null float64
노인사고발생건수      174 non-null float64
어린이사고발생건수     174 non-null float64
dtypes: float64(19), object(2)
memory usage: 29.9+ KB


In [326]:
del final_in["index"]

In [327]:
final_na_sumi = final_in.isnull().sum()
print(final_na_sumi)

구              0
new동           0
공시지가합계         0
공시지가평균         0
소인(외국인)       36
소인(외국인 비율)    36
소인(한국인)       36
소인(한국인 비율)    36
소인수           36
소인비율          36
노인수           36
노인비율          36
노인(한국인)       36
노인(한국인 비율)    36
노인(외국인)       36
노인(외국인 비율)    36
순수익           36
보행사고발생건수       0
노인사고발생건수       0
어린이사고발생건수      0
dtype: int64


In [328]:
Total.to_csv('final/final_in.csv', encoding="euc-kr")